# Ensemble Learning

## Initial Imports

In [1]:
        import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.ensemble import BalancedRandomForestClassifier 
from imblearn.ensemble import EasyEnsembleClassifier 
from sklearn.preprocessing import LabelEncoder

## Read the CSV and Perform Basic Data Cleaning

## Split the Data into Training and Testing

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [22]:
# Create our features
# X=df.copy()
X = df.drop(columns=['loan_status','issue_d', "pymnt_plan", "initial_list_status", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "collections_12_mths_ex_med", "policy_code", "acc_now_delinq", "chargeoff_within_12_mths", "delinq_amnt", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m", "tax_liens", "hardship_flag", "debt_settlement_flag",'next_pymnt_d', 'pub_rec'])


X = pd.get_dummies(X, columns=['home_ownership', 'verification_status','application_type'])

# Create our target
le = LabelEncoder()
le.fit(df['loan_status'])
df['loan_status'] = le.transform(df['loan_status'])
y = df['loan_status'].values.reshape(-1,1)

In [21]:
X.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,revol_bal,total_acc,...,total_il_high_credit_limit,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,application_type_Individual,application_type_Joint App
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,1609.0,14.0,...,61987.0,0,0,0,1,0,1,0,1,0
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,18368.0,34.0,...,49197.0,0,1,0,0,0,0,1,1,0
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,13247.0,21.0,...,43144.0,0,1,0,0,0,0,1,1,0
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,17996.0,23.0,...,76506.0,0,0,0,1,0,0,1,1,0
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,9091.0,28.0,...,20000.0,0,1,0,0,1,0,0,1,0


In [23]:
# Check the balance of our target values
y[:5]

array([[1],
       [1],
       [1],
       [1],
       [1]])

In [24]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [25]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [26]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [27]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled  = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [28]:
# Resample the training data with the BalancedRandomForestClassifier
brf_model = BalancedRandomForestClassifier(random_state=1)

In [29]:
# Calculated the balanced accuracy score
brf_model = brf_model.fit(X_train_scaled, y_train)
predictions = brf_model.predict(X_test_scaled)

In [30]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns =["Predicted 0", "Predicted 1"]
)
acc_score = balanced_accuracy_score(y_test, predictions)
cm

array([[   59,    42],
       [ 3467, 13637]])

In [31]:
print(f"Accuracy Score : {acc_score}")


Accuracy Score : 0.690728646648575


In [32]:
# Print the imbalanced classification report

print("Classification Report")
print(classification_report_imbalanced(y_test, predictions))

Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.02      0.58      0.80      0.03      0.68      0.46       101
          1       1.00      0.80      0.58      0.89      0.68      0.48     17104

avg / total       0.99      0.80      0.59      0.88      0.68      0.48     17205



In [33]:
# List the features sorted in descending order by feature importance
importances=brf_model.feature_importances_
sorted_importances = sorted(zip(brf_model.feature_importances_, X.columns), reverse=True)
sorted_importances[:10]

[(0.0729556796047547, 'total_rec_prncp'),
 (0.06461906810259042, 'last_pymnt_amnt'),
 (0.06274831133551759, 'total_pymnt_inv'),
 (0.05853021864178123, 'total_rec_int'),
 (0.04601881647768405, 'total_pymnt'),
 (0.03487309921486423, 'int_rate'),
 (0.020067790917659686, 'installment'),
 (0.01814915127495009, 'dti'),
 (0.01798253218631068, 'mo_sin_old_rev_tl_op'),
 (0.0174212920287605, 'out_prncp_inv')]

### Easy Ensemble Classifier

In [34]:
# Train the Classifier
ee_model = EasyEnsembleClassifier(random_state=1)
ee_model = ee_model.fit(X_train_scaled, y_train)
ee_predictions = brf_model.predict(X_test_scaled)

In [35]:
# Calculated the balanced accuracy score
cm = confusion_matrix(y_test, ee_predictions)
ee_acc_score = balanced_accuracy_score(y_test, ee_predictions)
print(f"Accuracy Score : {ee_acc_score}")

Accuracy Score : 0.690728646648575


In [36]:
# Display the confusion matrix
cm = confusion_matrix(y_test, ee_predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns =["Predicted 0", "Predicted 1"]
)
cm

array([[   59,    42],
       [ 3467, 13637]])

In [37]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report_imbalanced(y_test, ee_predictions))

Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.02      0.58      0.80      0.03      0.68      0.46       101
          1       1.00      0.80      0.58      0.89      0.68      0.48     17104

avg / total       0.99      0.80      0.59      0.88      0.68      0.48     17205



### Final Questions

1. Which model had the best balanced accuracy score?

   The balanced accuracy score for both models were the same. I do not think this is meant to be the same but not sure what is going on with the different classifiers. Maybe my variables are being overwritten?

2. Which model had the best recall score?

    SAME

3. Which model had the best geometric mean score?

    SAME

4. What are the top three features?

    Top 3 features were loan_status_low_risk, loan_status_high_risk and total_pymnt_inv